In [235]:
from backtesting.test import SMA
from backtesting.lib import SignalStrategy, TrailingStrategy
import pandas as pd
from backtesting.test import GOOG
import numpy as np

In [236]:
class SmaCross(SignalStrategy,
               TrailingStrategy):
    n1 = 10
    n2 = 25
    
    def init(self):
        # In init() and in next() it is important to call the
        # super method to properly initialize the parent classes
        super().init()
        
        # Precompute the two moving averages
        sma1 = self.I(SMA, self.data.Close, self.n1)
        sma2 = self.I(SMA, self.data.Close, self.n2)
        
        # Where sma1 crosses sma2 upwards. Diff gives us [-1,0, *1*]
        signal = (pd.Series(sma1) > sma2).astype(int).diff().fillna(0)
        signal = signal.replace(-1, 0)  # Upwards/long only
        
        # Use 95% of available liquidity (at the time) on each order.
        # (Leaving a value of 1. would instead buy a single share.)
        entry_size = signal * .95
                
        # `SignalStrategy`. See the docs.
        self.set_signal(entry_size=entry_size)
        
        # Set trailing stop-loss to 2x ATR using
        # the method provided by `TrailingStrategy`
        self.set_trailing_sl(2)

In [238]:
data = pd.read_csv("../data/clean/bitcoin_prices.csv")

data["Time"] = pd.to_datetime(data['Time'],unit='s')
data.set_index(pd.DatetimeIndex(data['Time']), inplace=True, drop=True)

data = data[:45999] 

In [239]:
from backtesting import Backtest
from backtesting.test import GOOG

bt = Backtest(data, SmaCross, commission=.002, cash=100000)

run_results = bt.run()
print(run_results)
bt.plot()

/Users/jmcallister/.local/share/virtualenvs/quant-trading-gcHAf6Sx/lib/python3.9/site-packages/backtesting/backtesting.py:1042: FutureWarning: Index.is_all_dates is deprecated, will be removed in a future version.  check index.inferred_type instead
  if (not data.index.is_all_dates and
/Users/jmcallister/.local/share/virtualenvs/quant-trading-gcHAf6Sx/lib/python3.9/site-packages/backtesting/backtesting.py:1057: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  if len(data.columns & {'Open', 'High', 'Low', 'Close', 'Volume'}) != 5:


Start                     2015-07-20 21:00:00
End                       2020-10-18 11:00:00
Duration                   1916 days 14:00:00
Exposure Time [%]                   58.081697
Equity Final [$]               2059329.301160
Equity Peak [$]                2456695.142900
Return [%]                        1959.329301
Buy & Hold Return [%]             4011.136691
Return (Ann.) [%]                   77.830058
Volatility (Ann.) [%]               86.232424
Sharpe Ratio                         0.902561
Sortino Ratio                        2.721693
Calmar Ratio                         2.039098
Max. Drawdown [%]                  -38.168870
Avg. Drawdown [%]                   -2.943377
Max. Drawdown Duration      489 days 04:00:00
Avg. Drawdown Duration        8 days 03:00:00
# Trades                                  694
Win Rate [%]                        39.481268
Best Trade [%]                      87.073537
Worst Trade [%]                    -16.244785
Avg. Trade [%]                    

/Users/jmcallister/.local/share/virtualenvs/quant-trading-gcHAf6Sx/lib/python3.9/site-packages/backtesting/_plotting.py:104: UserWarning: Data contains too many candlesticks to plot; downsampling to '8H'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "
